In [ ]:
# скачивание и распаковка проекта, распаковка данных
!wget https://github.com/gimaevra94/gan/archive/refs/heads/main.zip
!unzip /content/main.zip
!unzip /content/gan-main/data.zip

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from tqdm import tqdm_notebook as tqdm

class Gan(tf.keras.Model):

    def __init__(self):
        super().__init__()

        # лосс, инициализатор весов, оптимайзер, генератор, дискриминатор
        self.loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
        self.init=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.02)
        self.opti=tf.keras.optimizers.legacy.Adam(0.0002,beta_1=0.5,beta_2=0.999)
        self.gen=self.gen()
        self.disc=self.disc()

    def loss_gen(self,output_fake):
        """считает ошибку на основе фейкового выхода дискриминатора и метки класса 1"""
        return self.loss(tf.ones_like(output_fake),output_fake)

    def loss_disc(self,output_real,output_fake):
        """считает ошибку на основе фейкового и реального выходов дискриминатора и метками классов 1,0"""
        loss_real=self.loss(tf.ones_like(output_real),output_real)
        loss_fake=self.loss(tf.zeros_like(output_fake),output_fake)
        return loss_real+loss_fake

    def gen(self):
        """принимает вектор из нормального распределения и превращает его в размерность 80,80,1"""
        model=tf.keras.Sequential([
            tf.keras.layers.Reshape(target_shape=(1,1,100),input_shape=(100,)),

            tf.keras.layers.UpSampling2D(10),
            tf.keras.layers.Conv2D(256,
                                   kernel_size=4,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer=self.init),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(128,
                                   kernel_size=4,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer=self.init),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(64,
                                   kernel_size=4,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer=self.init),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(1,
                                   kernel_size=4,
                                   strides=1,
                                   activation="tanh",
                                   padding="same",
                                   kernel_initializer=self.init)])
        return model

    def disc(self):
        """принимает реальное изображение либо сгенерированное
        превращает его в вектор логитов"""
        model=tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=((80,80,1))),

            tf.keras.layers.Conv2D(64,
                                   kernel_size=4,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer=self.init,
                                   strides=2),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.Conv2D(128,
                                   kernel_size=4,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer=self.init,
                                   strides=2),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.Conv2D(256,
                                   kernel_size=4,
                                   padding="same",
                                   use_bias=False,
                                   kernel_initializer=self.init,
                                   strides=2),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.Conv2D(1,kernel_size=4,kernel_initializer=self.init),

            tf.keras.layers.Flatten()])

        return model

    @tf.function
    def train_step(self,imgs_real):
        """1. генератор принимает вектор нормального распределения и выдает фейк.данные
           2. дискриминатор принимает реальные данные и фейк.данные генератора
           3. выход дискриминатора на реальных и фейк.данных едет в loss_disc
           4. выход дискриминатора на фейк.данных едет в loss_gen
           5. с помощью лосса генератора и обученных параметров генератора
           считается градиент для генератора. аналогично для дискриминатора
           6. оба градиента передаются в оптимайзер"""
        random_normal_vector=tf.random.normal([tf.cast(imgs_real.shape[0],tf.int32),100])

        with tf.GradientTape() as gen_tape,tf.GradientTape() as disc_tape:
            imgs_generated=self.gen(random_normal_vector,training=True)

            output_real=self.disc(imgs_real,training=True)
            output_fake=self.disc(imgs_generated,training=True)

            loss_gen=self.loss_gen(output_fake)
            loss_disc=self.loss_disc(output_real,output_fake)

        grads_of_gen=gen_tape.gradient(loss_gen,self.gen.trainable_variables)
        grads_of_disc=disc_tape.gradient(loss_disc,self.disc.trainable_variables)

        self.opti.apply_gradients(zip(grads_of_gen,self.gen.trainable_variables))
        self.opti.apply_gradients(zip(grads_of_disc,self.disc.trainable_variables))

    def train(self,data):
        """каждую итерацию один батч реальных данных едет на вход дискриминатора"""
        for epoch in range(100):
            for batch in tqdm(data):
                self.train_step(batch)

gan=Gan()

path='content/'

data=pathlib.Path(path).parent/'data'
data2=data/'data2'
data3=data2/'data3'

train=tf.keras.utils.image_dataset_from_directory(directory=data2,
                                                  label_mode=None,
                                                  batch_size=256,
                                                  color_mode='grayscale',
                                                  image_size=(80,80))

rescale=tf.keras.layers.Rescaling(1./127.5,offset=-1)
train=train.map(lambda x:rescale(x))

gan.train(train)